### Bank Loan return Prediction System Using Pipeline

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline

In [33]:
data=pd.read_excel('C:/Users/cs04/Python_Programming/loan-data.xlsx')
data

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,...,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
0,2,2,2,0.0,0.0,0.00000,36 months,0.00,0.00,< 1 year,...,1.0,0.000000,0.00,0.00,0.00,2023-01-07,0.00,2023-01-07,2023-01-07,1
1,3,545583,703644,2500.0,2500.0,2500.00000,36 months,13.98,85.42,4 years,...,10.0,3075.291779,3075.29,2500.00,575.29,2023-07-13,90.85,2023-08-13,2023-06-16,0
2,4,532101,687836,5000.0,5000.0,5000.00000,36 months,15.95,175.67,4 years,...,15.0,2948.760000,2948.76,1909.02,873.81,2023-11-11,175.67,NaT,2023-03-12,1
3,5,877788,1092507,7000.0,7000.0,7000.00000,36 months,9.91,225.58,10+ years,...,20.0,8082.391880,8082.39,7000.00,1082.39,2023-03-14,1550.27,NaT,2023-03-14,0
4,6,875406,1089981,2000.0,2000.0,2000.00000,36 months,5.42,60.32,10+ years,...,15.0,2161.663244,2161.66,2000.00,161.66,2023-02-14,53.12,NaT,2023-06-16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38475,38476,849205,1060907,3000.0,3000.0,3000.00000,36 months,11.99,99.63,3 years,...,22.0,3586.619764,3586.62,3000.00,586.62,2023-08-14,100.30,NaT,2023-06-16,0
38476,38477,852914,1065048,10400.0,10400.0,10400.00000,36 months,13.49,352.88,9 years,...,33.0,12703.534030,12703.53,10400.00,2303.53,2023-09-14,393.08,NaT,2023-06-16,0
38477,38478,519553,671637,16000.0,10550.0,10531.35818,60 months,14.96,250.77,10+ years,...,25.0,14202.267530,14163.31,10550.00,3652.27,2023-06-13,5439.96,NaT,2023-02-16,0
38478,38479,825638,1034448,10000.0,10000.0,10000.00000,36 months,16.89,355.99,1 year,...,12.0,12815.178320,12815.18,10000.00,2815.18,2023-08-14,380.63,NaT,2023-09-15,0


In [34]:
#selection of Important or necessory columns
data=data[['funded_amnt','funded_amnt_inv','int_rate','term','installment','emp_length','home_ownership','annual_inc','verification_status','loan_status','purpose','last_pymnt_amnt','total_pymnt','total_rec_prncp','total_rec_int','repay_fail']]

In [35]:
data.head(3)

,funded_amnt,funded_amnt_inv,int_rate,term,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,last_pymnt_amnt,total_pymnt,total_rec_prncp,total_rec_int,repay_fail
0,0.0,0.0,0.00,36 months,0.00,< 1 year,RENT,0.0,Not Verified,Charged Off,major_purchase,0.00,0.000000,0.00,0.00,1
1,2500.0,2500.0,13.98,36 months,85.42,4 years,RENT,20004.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,90.85,3075.291779,2500.00,575.29,0
2,5000.0,5000.0,15.95,36 months,175.67,4 years,RENT,59000.0,Not Verified,Charged Off,debt_consolidation,175.67,2948.760000,1909.02,873.81,1


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38480 entries, 0 to 38479
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   funded_amnt          38479 non-null  float64
 1   funded_amnt_inv      38479 non-null  float64
 2   int_rate             38480 non-null  float64
 3   term                 38480 non-null  object 
 4   installment          38479 non-null  float64
 5   emp_length           37487 non-null  object 
 6   home_ownership       38480 non-null  object 
 7   annual_inc           38478 non-null  float64
 8   verification_status  38480 non-null  object 
 9   loan_status          38480 non-null  object 
 10  purpose              38480 non-null  object 
 11  last_pymnt_amnt      38479 non-null  float64
 12  total_pymnt          38479 non-null  float64
 13  total_rec_prncp      38479 non-null  float64
 14  total_rec_int        38479 non-null  float64
 15  repay_fail           38480 non-null 

In [37]:
# Checking For Null Values
data.isnull().sum()

funded_amnt              1
funded_amnt_inv          1
int_rate                 0
term                     0
installment              1
emp_length             993
home_ownership           0
annual_inc               2
verification_status      0
loan_status              0
purpose                  0
last_pymnt_amnt          1
total_pymnt              1
total_rec_prncp          1
total_rec_int            1
repay_fail               0
dtype: int64

In [38]:
# Dropping Null Values from Data Set
data.dropna(inplace=True)

C:\Users\cs04\AppData\Local\Temp\ipykernel_13632\3477753364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [39]:
# All null values are removed from Dataset
data.isnull().sum()

funded_amnt            0
funded_amnt_inv        0
int_rate               0
term                   0
installment            0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
purpose                0
last_pymnt_amnt        0
total_pymnt            0
total_rec_prncp        0
total_rec_int          0
repay_fail             0
dtype: int64

In [40]:
#checking for duplicate values
data.duplicated().sum()

0

In [41]:
data.describe()

,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,last_pymnt_amnt,total_pymnt,total_rec_prncp,total_rec_int,repay_fail
count,37485.000000,37485.000000,37485.000000,37485.000000,3.748500e+04,37485.000000,37485.000000,37485.000000,37485.000000,37485.000000
mean,10897.022142,10200.987254,12.193902,325.271064,6.960518e+04,2635.876489,12064.044329,9712.539042,2249.454463,0.149633
std,7153.932750,7142.134667,3.700471,209.370947,6.493932e+04,4412.045179,9023.234902,7064.398800,2577.358735,0.356716
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5275.000000,4975.000000,9.630000,167.340000,4.100000e+04,214.990000,5514.566072,4500.000000,669.030000,0.000000
50%,9600.000000,8575.000000,11.990000,280.170000,6.000000e+04,532.410000,9771.918186,8000.000000,1353.110000,0.000000
75%,15000.000000,14125.000000,14.740000,432.340000,8.300000e+04,3210.920000,16487.163130,13543.880000,2817.650000,0.000000
max,35000.000000,35000.000000,24.110000,1305.190000,6.000000e+06,36115.200000,58563.679930,35000.020000,23611.100000,1.000000


In [42]:
#Updating the Emp_length column to remove non mumeric values
data['emp_length']=data.emp_length.str.replace('<','').str.replace('+','').str.replace('year','').str.replace('s','').astype('int')
data['emp_length']

C:\Users\cs04\AppData\Local\Temp\ipykernel_13632\3648710673.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['emp_length']=data.emp_length.str.replace('<','').str.replace('+','').str.replace('year','').str.replace('s','').astype('int')
C:\Users\cs04\AppData\Local\Temp\ipykernel_13632\3648710673.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['emp_length']=data.emp_length.str.replace('<','').str.replace('+','').str.replace('year','').str.replace('s','').astype('int')


0         1
1         4
2         4
3        10
4        10
         ..
38475     3
38476     9
38477    10
38478     1
38479     7
Name: emp_length, Length: 37485, dtype: int32

In [43]:
data.head(3)

,funded_amnt,funded_amnt_inv,int_rate,term,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,last_pymnt_amnt,total_pymnt,total_rec_prncp,total_rec_int,repay_fail
0,0.0,0.0,0.00,36 months,0.00,1,RENT,0.0,Not Verified,Charged Off,major_purchase,0.00,0.000000,0.00,0.00,1
1,2500.0,2500.0,13.98,36 months,85.42,4,RENT,20004.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,90.85,3075.291779,2500.00,575.29,0
2,5000.0,5000.0,15.95,36 months,175.67,4,RENT,59000.0,Not Verified,Charged Off,debt_consolidation,175.67,2948.760000,1909.02,873.81,1


In [44]:
data.term=data.term.str.replace('months','').astype('int')

C:\Users\cs04\AppData\Local\Temp\ipykernel_13632\3660538848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.term=data.term.str.replace('months','').astype('int')


In [45]:
data['home_ownership']=data['home_ownership'].apply(lambda x: 'OTHER' if x=='NONE' else x)

C:\Users\cs04\AppData\Local\Temp\ipykernel_13632\4227359612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['home_ownership']=data['home_ownership'].apply(lambda x: 'OTHER' if x=='NONE' else x)


In [46]:
data.home_ownership.value_counts()

RENT        17868
MORTGAGE    16723
OWN          2765
OTHER         129
Name: home_ownership, dtype: int64

In [47]:
data['verification_status']=data['verification_status'].apply(lambda x: 'Verified' if x=='Source Verified' else x)

C:\Users\cs04\AppData\Local\Temp\ipykernel_13632\1648076954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['verification_status']=data['verification_status'].apply(lambda x: 'Verified' if x=='Source Verified' else x)


In [48]:
data.verification_status.value_counts()

Verified        20925
Not Verified    16560
Name: verification_status, dtype: int64

In [49]:
data

,funded_amnt,funded_amnt_inv,int_rate,term,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,last_pymnt_amnt,total_pymnt,total_rec_prncp,total_rec_int,repay_fail
0,0.0,0.00000,0.00,36,0.00,1,RENT,0.0,Not Verified,Charged Off,major_purchase,0.00,0.000000,0.00,0.00,1
1,2500.0,2500.00000,13.98,36,85.42,4,RENT,20004.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,90.85,3075.291779,2500.00,575.29,0
2,5000.0,5000.00000,15.95,36,175.67,4,RENT,59000.0,Not Verified,Charged Off,debt_consolidation,175.67,2948.760000,1909.02,873.81,1
3,7000.0,7000.00000,9.91,36,225.58,10,MORTGAGE,53796.0,Not Verified,Fully Paid,other,1550.27,8082.391880,7000.00,1082.39,0
4,2000.0,2000.00000,5.42,36,60.32,10,RENT,30000.0,Not Verified,Fully Paid,debt_consolidation,53.12,2161.663244,2000.00,161.66,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38475,3000.0,3000.00000,11.99,36,99.63,3,RENT,74250.0,Verified,Fully Paid,credit_card,100.30,3586.619764,3000.00,586.62,0
38476,10400.0,10400.00000,13.49,36,352.88,9,RENT,62000.0,Verified,Fully Paid,credit_card,393.08,12703.534030,10400.00,2303.53,0
38477,10550.0,10531.35818,14.96,60,250.77,10,MORTGAGE,95088.0,Verified,Fully Paid,debt_consolidation,5439.96,14202.267530,10550.00,3652.27,0
38478,10000.0,10000.00000,16.89,36,355.99,1,RENT,48720.0,Not Verified,Fully Paid,debt_consolidation,380.63,12815.178320,10000.00,2815.18,0


In [50]:
data.head(3)

,funded_amnt,funded_amnt_inv,int_rate,term,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,last_pymnt_amnt,total_pymnt,total_rec_prncp,total_rec_int,repay_fail
0,0.0,0.0,0.00,36,0.00,1,RENT,0.0,Not Verified,Charged Off,major_purchase,0.00,0.000000,0.00,0.00,1
1,2500.0,2500.0,13.98,36,85.42,4,RENT,20004.0,Not Verified,Does not meet the credit policy. Status:Fully ...,other,90.85,3075.291779,2500.00,575.29,0
2,5000.0,5000.0,15.95,36,175.67,4,RENT,59000.0,Not Verified,Charged Off,debt_consolidation,175.67,2948.760000,1909.02,873.81,1


In [51]:
data.drop(columns='loan_status',inplace=True)

C:\Users\cs04\AppData\Local\Temp\ipykernel_13632\1317498484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns='loan_status',inplace=True)


In [52]:
data

,funded_amnt,funded_amnt_inv,int_rate,term,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,last_pymnt_amnt,total_pymnt,total_rec_prncp,total_rec_int,repay_fail
0,0.0,0.00000,0.00,36,0.00,1,RENT,0.0,Not Verified,major_purchase,0.00,0.000000,0.00,0.00,1
1,2500.0,2500.00000,13.98,36,85.42,4,RENT,20004.0,Not Verified,other,90.85,3075.291779,2500.00,575.29,0
2,5000.0,5000.00000,15.95,36,175.67,4,RENT,59000.0,Not Verified,debt_consolidation,175.67,2948.760000,1909.02,873.81,1
3,7000.0,7000.00000,9.91,36,225.58,10,MORTGAGE,53796.0,Not Verified,other,1550.27,8082.391880,7000.00,1082.39,0
4,2000.0,2000.00000,5.42,36,60.32,10,RENT,30000.0,Not Verified,debt_consolidation,53.12,2161.663244,2000.00,161.66,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38475,3000.0,3000.00000,11.99,36,99.63,3,RENT,74250.0,Verified,credit_card,100.30,3586.619764,3000.00,586.62,0
38476,10400.0,10400.00000,13.49,36,352.88,9,RENT,62000.0,Verified,credit_card,393.08,12703.534030,10400.00,2303.53,0
38477,10550.0,10531.35818,14.96,60,250.77,10,MORTGAGE,95088.0,Verified,debt_consolidation,5439.96,14202.267530,10550.00,3652.27,0
38478,10000.0,10000.00000,16.89,36,355.99,1,RENT,48720.0,Not Verified,debt_consolidation,380.63,12815.178320,10000.00,2815.18,0


In [53]:
# one hot encoding
trf1 = ColumnTransformer([
    ('performing_encoding',OneHotEncoder(sparse=False,handle_unknown='ignore'),[6,8,9])
],remainder='passthrough')

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score,accuracy_score,classification_report
from sklearn.tree import DecisionTreeClassifier

In [56]:
# train the model
trf2 = LogisticRegression()

## Create Pipeline

In [57]:
pipe = Pipeline([
                ('trf1',trf1),
                ('trf2',trf2)
            ])

In [58]:
y=data['repay_fail']
y

0        1
1        0
2        1
3        0
4        0
        ..
38475    0
38476    0
38477    0
38478    0
38479    0
Name: repay_fail, Length: 37485, dtype: int64

In [59]:
X=data.drop(columns='repay_fail')

In [60]:
X

,funded_amnt,funded_amnt_inv,int_rate,term,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,last_pymnt_amnt,total_pymnt,total_rec_prncp,total_rec_int
0,0.0,0.00000,0.00,36,0.00,1,RENT,0.0,Not Verified,major_purchase,0.00,0.000000,0.00,0.00
1,2500.0,2500.00000,13.98,36,85.42,4,RENT,20004.0,Not Verified,other,90.85,3075.291779,2500.00,575.29
2,5000.0,5000.00000,15.95,36,175.67,4,RENT,59000.0,Not Verified,debt_consolidation,175.67,2948.760000,1909.02,873.81
3,7000.0,7000.00000,9.91,36,225.58,10,MORTGAGE,53796.0,Not Verified,other,1550.27,8082.391880,7000.00,1082.39
4,2000.0,2000.00000,5.42,36,60.32,10,RENT,30000.0,Not Verified,debt_consolidation,53.12,2161.663244,2000.00,161.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38475,3000.0,3000.00000,11.99,36,99.63,3,RENT,74250.0,Verified,credit_card,100.30,3586.619764,3000.00,586.62
38476,10400.0,10400.00000,13.49,36,352.88,9,RENT,62000.0,Verified,credit_card,393.08,12703.534030,10400.00,2303.53
38477,10550.0,10531.35818,14.96,60,250.77,10,MORTGAGE,95088.0,Verified,debt_consolidation,5439.96,14202.267530,10550.00,3652.27
38478,10000.0,10000.00000,16.89,36,355.99,1,RENT,48720.0,Not Verified,debt_consolidation,380.63,12815.178320,10000.00,2815.18


In [61]:
from sklearn.model_selection import train_test_split

In [62]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [63]:
# train
pipe.fit(X_train,y_train)

C:\Users\cs04\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\cs04\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('performing_encoding',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [6, 8, 9])])),
                ('trf2', LogisticRegression())])

In [64]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('performing_encoding',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [6, 8, 9])]),
 'trf2': LogisticRegression()}

In [65]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('performing_encoding',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [6, 8, 9])])),
                ('trf2', LogisticRegression())])

In [66]:
# Predict
y_pred = pipe.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [67]:
from sklearn.metrics import accuracy_score,r2_score,classification_report

#Accuracy of Model
accuracy_score(y_test,y_pred)

0.9971988795518207

In [68]:
score=r2_score(y_test,y_pred)
score

0.978133586001895

In [69]:
report=classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6366
           1       1.00      0.98      0.99      1131

    accuracy                           1.00      7497
   macro avg       1.00      0.99      0.99      7497
weighted avg       1.00      1.00      1.00      7497

